# Summarization on Custom Dataset with SageMaker Jumpstart and [LangChain](https://python.langchain.com/en/latest/index.html) Library

Reference: https://github.com/gkamradt/langchain-tutorials/tree/main/data_generation


 There are two main types of methods for summarizing text: abstractive and extractive.

Abstractive summarization generates a new shorter summary in its own words based on understanding the meaning and concepts of the original text. It analyzes the text using advanced natural language techniques to grasp the key ideas and then expresses those ideas in a summarized form using different words and phrases. This is similar to how humans summarize by reading something and then explaining the main points in their own words.

Extractive summarization works by selecting the most important sentences, phrases or words from the original text to construct a summary. It calculates the weight or importance of each part of the text using algorithms and then chooses the parts with the highest weights to put into the summary. This pulls summarizes by extracting key elements from the text itself rather than interpreting the meaning.

So in short, abstractive summarization rewrites the key ideas in new words while extractive summarization selects the most salient parts of the existing text. Both aim to distill the essence and most significant information from the original document into a condensed summary.

We're going to run through 3 methods for summarization that start with basic prompting to summarizing large documents using `map_reduce` method. These aren't the only options, feel free to modify it based on your use case. 

**3 Levels Of Summarization:**
1. **Summarize a couple sentences** - Basic Prompt
2. **Summarize a couple paragraphs** - Prompt Templates
3. **Summarize a large document with multiple pages** - Map Reduce
4. **Summarize a book**

In this notebook we will demonstrate how to use **AI21 Summary API** for text summarization using a library of documents as a reference.

**This notebook serves a template such that you can easily replace the example dataset by your own to build a custom text summarization application.**

## Deploy large language model (LLM) and embedding model in SageMaker JumpStart

Make sure to deploy the ai21 summary model from jumpstart before you begin following the notebook and provide the endpoint here. You can do this by subscribing to the AI21 Summarize model, then clicking on `Open Notebook` option. This will open the notebook in Amazon SageMaker Studio. Run through the notebook to deploy the model, capture the endpoint name and return to this notebook. 

In [ ]:
!pip install --upgrade sagemaker
!pip install ipywidgets==7.0.0
!pip install langchain
!pip install faiss-cpu 
!pip install pytesseract
!pip install unstructured
!pip install transformers
!pip install pypdf
!pip install langchain-community

In [ ]:
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase
from langchain import PromptTemplate

# 明确指定一个区域，例如 'us-west-2'
region_name = 'us-west-2'  # 您可以选择任何在上面列表中的可用区域

# 创建带有指定区域的 boto3 会话
boto_session = boto3.Session(region_name=region_name)

# 使用这个 boto3 会话创建 SageMaker 会话
sagemaker_session = Session(boto_session=boto_session)

# 获取 AWS 角色和区域
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto_session.region_name

# 创建 SageMaker 会话
sess = sagemaker.Session(boto_session=boto_session)

# 其他变量设置
model_version = "*"
endpoint_name = 'summarize'  # 替换为您的端点名称

# 打印确认信息
print(f"Using AWS region: {aws_region}")
print(f"AWS role: {aws_role}")

## Summarize couple of sentences 

In [ ]:
prompt = """
Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""

Next, we wrap up our SageMaker endpoints for LLM into `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. 

In [ ]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
import json

class ContentHandler(LLMContentHandler):
    '''
    自定义的内容处理器类，用于处理与SageMaker端点之间的输入和输出。
    这个类继承自LLMContentHandler，专门用于处理特定格式的JSON数据。
    '''
    
    # 指定内容类型为JSON
    content_type = "application/json"
    # 指定接受的响应类型也为JSON
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        将输入提示转换为SageMaker端点可接受的格式。
        
        参数:
        prompt (str): 输入的文本提示
        model_kwargs (dict): 额外的模型参数（这里未使用）
        
        返回:
        bytes: 编码后的JSON字符串
        '''
        # 创建包含源文本和类型的JSON对象
        input_str = json.dumps({
            "source": prompt,
            "sourceType": "TEXT"
        })
        # 将JSON字符串编码为UTF-8字节
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        '''
        将SageMaker端点的输出转换为所需的格式。
        
        参数:
        output (bytes): 从SageMaker端点接收的原始字节输出
        
        返回:
        str: 提取的摘要文本
        '''
        # 解码输出并解析JSON
        response_json = json.loads(output.read().decode("utf-8"))
        # 返回JSON中的'summary'字段
        return response_json["summary"]

# 创建ContentHandler实例
content_handler = ContentHandler()

# 创建SagemakerEndpoint实例，用于与SageMaker端点交互
sm_llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,  # 使用预定义的端点名称
    region_name=aws_region,       # 使用预定义的AWS区域
    # model_kwargs=parameters,    # 可选：额外的模型参数（当前被注释掉）
    content_handler=content_handler,  # 使用自定义的内容处理器
)

# 注意：endpoint_name 和 aws_region 应该在之前的代码中定义
# 这个SagemakerEndpoint实例现在可以用于发送请求到指定的SageMaker端点

In [ ]:
# 使用SageMaker端点LLM计算提示中的token数量
num_tokens = sm_llm.get_num_tokens(prompt)

# 打印计算得到的token数量
print(f"Our prompt has {num_tokens} tokens")

'''
解释：

1. sm_llm.get_num_tokens(prompt):
   - 这个方法调用SageMaker端点来计算给定提示（prompt）中的token数量。
   - token是文本的基本单位，通常是单词或子词。不同的模型可能有不同的tokenization方法。

2. 计算token数量的重要性：
   - 了解提示的token数量对于管理输入大小很重要，因为许多模型有最大输入长度限制。
   - 它也可以帮助估计API调用的成本，因为许多服务按token数量计费。

3. 变量说明：
   - prompt: 应该是之前定义的输入文本。
   - num_tokens: 存储计算得到的token数量。

4. 打印结果：
   - 使用f-string格式化输出，显示提示中包含的token数量。
   - 这对于调试和优化提示长度很有用。

注意：确保在运行此代码之前，prompt变量已经被定义，并且sm_llm已经正确初始化和配置。
'''

In [ ]:
# 使用SageMaker端点LLM处理提示并获取输出
output = sm_llm(prompt)

# 打印生成的输出
print(output)

'''
解释：

1. sm_llm(prompt):
   - 这行代码调用了之前创建的SagemakerEndpoint实例（sm_llm）。
   - 它将prompt作为输入发送到SageMaker端点。
   - 这个调用会触发以下过程：
     a. 使用ContentHandler的transform_input方法处理输入。
     b. 发送处理后的输入到SageMaker端点。
     c. 接收端点的响应。
     d. 使用ContentHandler的transform_output方法处理响应。

2. 输出处理：
   - 端点返回的输出（可能是摘要或其他生成的文本）被赋值给output变量。
   - 根据之前定义的ContentHandler，输出应该是JSON响应中的"summary"字段。

3. print(output):
   - 直接打印处理后的输出。
   - 这可能是一个文本摘要，具体取决于SageMaker端点模型的功能。

4. 注意事项：
   - 确保prompt变量包含有效的输入文本。
   - 输出的具体内容和格式取决于SageMaker端点上部署的模型。
   - 如果输出很长，可能需要考虑更复杂的显示方法，比如分段打印或保存到文件。

5. 错误处理：
   - 这个简单的调用没有包含错误处理。在生产环境中，应该添加try-except块来处理可能的异常。

6. 性能考虑：
   - 对于长文本或批量处理，可能需要考虑异步调用或批处理方法以提高效率。
'''

In [ ]:
prompt = """
Write a ~ 1 sentence summary of the following text:

TEXT:
Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""

In [ ]:
output = sm_llm(prompt)
print (output)

##  Summarize a couple paragraphs -  Prompt Templates

Prompt templates are a great way to dynamically place text within your prompts. They are like [python f-strings](https://realpython.com/python-f-strings/) but specialized for working with language models.

We're going to look at 2 short Paul Graham essays

In [ ]:
# 定义包含Paul Graham文章文件路径的列表
paul_graham_essays = ['data/PaulGrahamEssaySmall/getideas.txt', 'data/PaulGrahamEssaySmall/noob.txt']

# 初始化一个空列表来存储文章内容
essays = []

# 遍历文件路径列表，读取每个文件的内容
for file_name in paul_graham_essays:
    with open(file_name, 'r') as file:
        essays.append(file.read())

'''
解释：

1. paul_graham_essays 列表：
   - 这个列表包含了两个文件的路径，这些文件是Paul Graham的文章。
   - 文件路径表明这些文章存储在一个名为'PaulGrahamEssaySmall'的目录中。

2. essays 列表：
   - 初始化为空列表，用于存储读取的文章内容。

3. for 循环：
   - 遍历 paul_graham_essays 列表中的每个文件路径。

4. with open(file_name, 'r') as file:
   - 使用 'with' 语句安全地打开文件，确保文件在使用后被正确关闭。
   - 'r' 模式表示以只读方式打开文件。

5. essays.append(file.read()):
   - file.read() 读取整个文件的内容。
   - append() 方法将读取的内容添加到 essays 列表中。

6. 结果：
   - 循环结束后，essays 列表将包含两篇文章的全文内容。
   - 每个列表项对应一篇完整的文章。

7. 注意事项：
   - 这种方法假设文件不是很大。对于大文件，可能需要考虑分块读取。
   - 确保文件路径是正确的，否则可能会引发 FileNotFoundError。
   - 文件应该是文本格式，且编码与系统默认编码兼容（通常是UTF-8）。

8. 潜在的改进：
   - 可以添加错误处理，比如处理文件不存在的情况。
   - 对于大量或大型文件，可以考虑使用生成器来逐个处理文件，而不是一次性加载所有内容。
'''

In [ ]:
# 遍历essays列表，打印每篇文章的编号和前300个字符
for i, essay in enumerate(essays):
    print(f"Essay #{i+1}: {essay[:300]}\n")

'''
解释：

1. for 循环和 enumerate():
   - enumerate(essays) 创建一个迭代器，它产生每篇文章的索引和内容。
   - i 是当前文章的索引（从0开始）。
   - essay 是当前文章的全文内容。

2. 打印格式：
   - f"Essay #{i+1}: ..." 使用f-string格式化输出。
   - i+1 用于显示从1开始的文章编号，而不是从0开始的索引。

3. essay[:300]:
   - 这是Python的切片操作，提取文章内容的前300个字符。
   - 如果文章少于300个字符，它会返回整个文章。

4. print() 函数：
   - 打印格式化的字符串。
   - \n 在每个打印输出的末尾添加一个空行，使输出更易读。

5. 输出结果：
   - 每篇文章都会显示其编号和开头的300个字符。
   - 这提供了每篇文章内容的快速预览。

6. 用途：
   - 这种方法对于快速检查加载的文章内容很有用。
   - 它可以帮助验证文件是否被正确读取，以及内容的大致情况。

7. 注意事项：
   - 如果文章包含非ASCII字符，确保你的环境支持正确显示这些字符。
   - 300个字符可能会在单词中间截断，这在某些情况下可能不理想。

8. 可能的改进：
   - 可以添加一个选项来自定义预览的字符数。
   - 可以实现更智能的截断，例如在最近的句子或段落结束处截断。
   - 对于非常长的文章列表，可以考虑只打印前几篇文章的预览。
'''

Next let's create a prompt template which will hold our instructions and a placeholder for the essay. In this example we only want a 1 sentence summary to come back

In [ ]:
from langchain import PromptTemplate

# 定义提示模板
template = """
Write a ~ 50 words summary of the following text:
{essay}
"""

# 创建PromptTemplate对象
prompt = PromptTemplate(
    input_variables=["essay"],
    template=template
)

'''
解释：

1. 导入 PromptTemplate:
   - 从 langchain 库导入 PromptTemplate 类，用于创建结构化的提示模板。

2. 定义模板字符串 (template):
   - 这是一个多行字符串，定义了提示的基本结构。
   - "{essay}" 是一个占位符，将在后续被实际的文章内容替换。
   - 模板要求生成大约50字的摘要。

3. 创建 PromptTemplate 对象:
   - input_variables=["essay"]: 指定模板中的变量。这里只有一个变量 "essay"。
   - template=template: 使用上面定义的模板字符串。

4. PromptTemplate 的作用:
   - 它提供了一种结构化和可重用的方式来创建提示。
   - 允许动态地插入变量（在这个例子中是文章内容）到预定义的模板中。

5. 使用方法:
   - 后续可以使用 prompt.format(essay=some_text) 来生成完整的提示。
   - 这将用实际的文章内容替换 {essay} 占位符。

6. 优点:
   - 提高代码的可读性和可维护性。
   - 使得批量处理多篇文章变得简单。
   - 允许轻松修改提示结构，而不需要更改主要的处理逻辑。

7. 注意事项:
   - 确保模板中的指令清晰且符合目标模型的能力。
   - "~50 words" 是一个近似值，实际生成的摘要长度可能会有所不同。

8. 潜在的扩展:
   - 可以添加更多的变量，如文章标题、作者等。
   - 可以创建多个模板用于不同的任务（如摘要、分析、问答等）。
'''

In [ ]:
# 遍历每篇文章，生成摘要
for essay in essays:
    # 使用之前定义的模板生成完整的提示
    summary_prompt = prompt.format(essay=essay)
    
    # 计算提示的token数量
    num_tokens = sm_llm.get_num_tokens(summary_prompt)
    print(f"This prompt + essay has {num_tokens} tokens")
    
    # 使用SageMaker端点生成摘要
    summary = sm_llm(summary_prompt)
    
    # 打印生成的摘要
    print(f"Summary: {summary.strip()}")
    print("\n")

'''
解释：

1. 遍历essays列表：
   - 对每篇文章单独处理。

2. 生成摘要提示：
   - prompt.format(essay=essay) 将当前文章内容插入到之前定义的模板中。
   - 这创建了一个完整的提示，包含指令和文章内容。

3. 计算token数量：
   - sm_llm.get_num_tokens(summary_prompt) 计算完整提示的token数。
   - 这有助于监控输入大小，确保不超过模型的最大输入限制。

4. 打印token数量：
   - 显示每个提示（包括文章）的token数，有助于了解输入规模。

5. 生成摘要：
   - sm_llm(summary_prompt) 调用SageMaker端点来处理提示并生成摘要。
   - 这个过程可能需要一些时间，取决于模型和文章长度。

6. 打印摘要：
   - summary.strip() 移除摘要开头和结尾的空白字符。
   - 打印格式化的摘要输出。

7. 添加空行：
   - print("\n") 在每个摘要后添加一个空行，提高可读性。

8. 注意事项：
   - 这个过程可能相对耗时，特别是对于长文章或多篇文章。
   - 确保SageMaker端点能够处理可能的大量请求。

9. 潜在的改进：
   - 可以添加错误处理，以应对可能的API调用失败。
   - 对于大量文章，考虑使用异步处理或批处理来提高效率。
   - 可以添加进度指示器，特别是处理大量文章时。
   - 考虑将结果保存到文件，而不仅仅是打印到控制台。

10. 性能和成本考虑：
    - 每次调用SageMaker端点都可能产生费用。
    - 对于大量文章，可能需要实现某种形式的节流或批处理机制。
'''

## Summarize a couple pages multiple pages - MapReduce

If you have multiple pages you'd like to summarize, you'll likely run into a token limit. Token limits won't always be a problem, but it is good to know how to handle them if you run into the issue.

The chain type "Map Reduce" is a method that helps with this. You first generate a summary of smaller chunks (that fit within the token limit) and then you get a summary of the summaries.\

Check out [this video](https://www.youtube.com/watch?v=f9_BWhCI4Zo) for more information on how chain types work.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

'''
解释：

1. 从langchain.chains.summarize导入load_summarize_chain：
   - load_summarize_chain 是一个用于创建文本摘要链的函数。
   - 摘要链是一系列处理步骤的组合，用于生成文本摘要。
   - 这个函数可以自动设置一个预配置的摘要流程，简化了摘要任务的实现。

2. 从langchain.text_splitter导入RecursiveCharacterTextSplitter：
   - RecursiveCharacterTextSplitter 是一个文本分割工具。
   - 它用于将长文本分割成更小的块，这在处理大型文档时特别有用。
   - "递归"意味着它可以智能地在特定字符（如段落分隔符）处分割文本。

3. 这些导入的用途：
   - load_summarize_chain 将用于创建一个自动化的摘要生成流程。
   - RecursiveCharacterTextSplitter 将用于预处理长文本，使其适合模型的输入限制。

4. 摘要链的优势：
   - 提供了一个结构化的方法来处理文档摘要。
   - 可以处理比单个模型调用更长的文档。
   - 允许更复杂的摘要策略，如先总结各部分再总结整体。

5. 文本分割器的重要性：
   - 允许处理超出模型最大输入长度的文档。
   - 可以保持文本的语义结构，如段落或句子的完整性。
   - 提高了处理长文档时的效率和准确性。

6. 后续步骤：
   - 通常，您会使用RecursiveCharacterTextSplitter来分割文本。
   - 然后，使用load_summarize_chain创建一个摘要链。
   - 最后，将分割后的文本输入到摘要链中生成摘要。

7. 注意事项：
   - 需要根据具体的文本特性和模型限制来配置文本分割器。
   - 摘要链的具体行为可能需要根据任务需求进行调整。

8. 潜在用途：
   - 处理长文档或书籍的摘要。
   - 创建分层摘要系统，如先摘要章节，再摘要整本书。
   - 实现更复杂的文档分析任务，如提取关键信息或生成报告。
'''

In [ ]:
# 定义Paul Graham文章的文件路径
paul_graham_essay = 'data/PaulGrahamEssays/startupideas.txt'

# 打开并读取文章内容
with open(paul_graham_essay, 'r') as file:
    essay = file.read()

'''
解释：

1. 文件路径定义：
   - paul_graham_essay 变量存储了Paul Graham的一篇文章的文件路径。
   - 路径 'data/PaulGrahamEssays/startupideas.txt' 表明这是一篇关于创业想法的文章。

2. 文件打开和读取：
   - 使用 with open(...) as file: 语句安全地打开文件。
   - 'r' 模式表示以只读方式打开文件。

3. 读取文件内容：
   - file.read() 读取整个文件的内容。
   - 文件内容被存储在 essay 变量中。

4. 上下文管理器（with语句）的作用：
   - 确保文件在使用后被正确关闭，即使发生异常也能保证关闭。
   - 这是处理文件的推荐方式，可以避免资源泄露。

5. 变量使用：
   - essay 变量现在包含了整篇文章的文本内容。
   - 这个变量可以在后续的处理中使用，如文本分析、摘要生成等。

6. 注意事项：
   - 确保文件路径是正确的，否则会引发 FileNotFoundError。
   - 这种方法假设文件不是很大。对于非常大的文件，可能需要考虑分块读取。
   - 确保文件是文本格式，且编码与系统默认编码兼容（通常是UTF-8）。

7. 潜在的改进：
   - 可以添加错误处理，比如处理文件不存在或权限问题的情况。
   - 对于大文件，可以考虑使用 for line in file: 逐行读取。
   - 可以指定文件编码，例如：open(paul_graham_essay, 'r', encoding='utf-8')

8. 后续使用：
   - 读取的内容可以用于文本分析、摘要生成、关键词提取等任务。
   - 可以打印 essay[:100] 来查看文章的开头，确认内容已正确读取。
'''

In [ ]:
# 计算文章的token数量
token_count = sm_llm.get_num_tokens(essay)

'''
解释：

1. sm_llm.get_num_tokens() 方法：
   - 这个方法是之前创建的SagemakerEndpoint实例（sm_llm）的一部分。
   - 它用于计算给定文本中的token数量。

2. 参数 essay：
   - 这是之前从文件中读取的整篇文章内容。
   - 传递给get_num_tokens方法进行token计数。

3. token的概念：
   - 在自然语言处理中，token通常指的是文本的基本单位。
   - 可能是单词、子词或字符，具体取决于模型的tokenization方法。

4. 计算token数量的目的：
   - 了解文本的长度，这对于管理模型输入很重要。
   - 许多语言模型有最大输入token限制。
   - 有助于估计处理文本可能需要的时间和资源。

5. 返回值：
   - 方法返回一个整数，表示文章中的token数量。
   - 这个值被赋给变量token_count（虽然在提供的代码片段中没有显示赋值）。

6. 注意事项：
   - token数量可能与单词数量不同，通常会更多。
   - 不同的模型可能有不同的tokenization方法，因此token数可能会有所不同。

7. 潜在用途：
   - 决定是否需要将文本分割成更小的部分。
   - 估算处理文本的成本（如果API按token收费）。
   - 判断文本是否适合一次性输入到模型中。

8. 可能的后续步骤：
   - 如果token数量超过模型的限制，可能需要使用文本分割器。
   - 可以打印token数量，以便了解文本的规模。

示例打印语句：
print(f"The essay contains {token_count} tokens.")
'''

That's too many, let's split our text up into chunks so they fit into the prompt limit. I'm going a chunk size of 10,000 characters. 

> You can think of tokens as pieces of words used for natural language processing. For English text, **1 token is approximately 4 characters** or 0.75 words. As a point of reference, the collected works of Shakespeare are about 900,000 words or 1.2M tokens.

This means the number of tokens we should expect is 10,000 / 4 = ~2,500 token chunks. But this will vary, each body of text/code will be different

In [ ]:
# 创建RecursiveCharacterTextSplitter实例
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], 
    chunk_size=10000, 
    chunk_overlap=500
)

# 使用文本分割器创建文档
docs = text_splitter.create_documents([essay])

'''
解释：

1. RecursiveCharacterTextSplitter:
   - 这是一个用于将长文本分割成更小块的工具。
   - "递归"意味着它会按照指定的分隔符列表逐级尝试分割文本。

2. 参数解释：
   - separators=["\n\n", "\n"]: 
     * 指定分割文本的分隔符。
     * 首先尝试用两个换行符（段落分隔）分割，如果块仍然太大，则使用单个换行符。
   - chunk_size=10000: 
     * 每个文本块的目标大小（以字符为单位）。
     * 分割器会尽量创建不超过这个大小的块。
   - chunk_overlap=500: 
     * 相邻块之间的重叠字符数。
     * 有助于保持上下文连贯性，特别是在块的边界处。

3. create_documents 方法：
   - 接受一个文本列表（这里只有一个文章），并返回一个文档对象列表。
   - 每个文档对象代表原始文本的一个块。

4. docs 变量：
   - 存储了分割后的文档对象列表。
   - 每个文档对象通常包含文本内容和可能的元数据。

5. 分割的重要性：
   - 允许处理超出模型最大输入长度的文档。
   - 有助于更精确地分析长文档的不同部分。

6. 注意事项：
   - chunk_size 应根据模型的最大输入长度和任务需求来设置。
   - chunk_overlap 有助于保持上下文，但会增加总处理量。

7. 潜在用途：
   - 为长文档生成摘要。
   - 对文档的不同部分进行独立分析。
   - 实现大型文档的问答系统。

8. 可能的后续步骤：
   - 遍历 docs 列表，对每个文档块单独处理。
   - 使用这些分割后的文档创建摘要链。

示例：打印分割后的文档数量
print(f"The essay has been split into {len(docs)} chunks.")
'''

In [ ]:
# 获取分割后的文档数量
num_docs = len(docs)

# 计算第一个文档的token数量
num_tokens_first_doc = sm_llm.get_num_tokens(docs[0].page_content)

# 打印文档数量和第一个文档的token数
print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

'''
解释：

1. 文档数量计算：
   - len(docs) 返回docs列表的长度，即分割后的文档数量。
   - 这个数量被存储在num_docs变量中。

2. 第一个文档的token计数：
   - docs[0] 访问分割后的第一个文档对象。
   - .page_content 获取该文档对象的文本内容。
   - sm_llm.get_num_tokens() 计算这段文本的token数量。
   - 结果存储在num_tokens_first_doc变量中。

3. sm_llm.get_num_tokens():
   - 这是之前定义的SageMaker端点LLM实例的方法。
   - 用于计算给定文本的token数量。

4. 打印语句：
   - 使用f-string格式化输出信息。
   - 显示总文档数和第一个文档的token数。

5. 这段代码的目的：
   - 验证文本分割的效果。
   - 确认分割后的文档大小是否符合预期。
   - 为后续处理提供基本信息。

6. 注意事项：
   - 文档数量应该与原文长度和chunk_size设置相符。
   - 第一个文档的token数应该接近（但不超过）之前设置的chunk_size。

7. 潜在的后续分析：
   - 可以遍历所有文档，计算每个文档的token数。
   - 检查是否有任何文档超出了预期的大小限制。
   - 根据需要调整RecursiveCharacterTextSplitter的参数。

8. 可能的改进：
   - 计算所有文档的平均token数。
   - 找出token数最多和最少的文档。
   - 可视化文档长度分布。

示例扩展：
avg_tokens = sum(sm_llm.get_num_tokens(doc.page_content) for doc in docs) / num_docs
print(f"Average tokens per document: {avg_tokens:.2f}")
'''

Great, assuming that number of tokens is consistent in the other docs we should be good to go. Let's use LangChain's [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) method, we will use `refine` chain type for summarization. We first need to initialize our chain

In [ ]:
# 创建摘要链
summary_chain = load_summarize_chain(
    llm=sm_llm, 
    chain_type='map_reduce',
    verbose=True  # 设置为True以查看使用的提示
)

'''
解释：

1. load_summarize_chain 函数：
   - 来自 langchain.chains.summarize 模块。
   - 用于创建一个自动化的文档摘要处理链。

2. 参数解释：
   - llm=sm_llm: 
     * 指定使用的语言模型。
     * 这里使用之前创建的SageMaker端点LLM实例。
   
   - chain_type='map_reduce':
     * 指定摘要链的类型为"map_reduce"。
     * 这种类型首先对每个文档块生成摘要（map步骤），然后将这些摘要组合成最终摘要（reduce步骤）。
     * 适合处理长文档或多个文档。

   - verbose=True:
     * 启用详细输出模式。
     * 将显示处理过程中使用的提示和中间结果，有助于调试和理解流程。

3. map_reduce 策略的工作原理：
   - Map: 对每个文档块单独生成摘要。
   - Reduce: 将所有单独的摘要合并成一个最终摘要。
   - 这种方法可以有效处理大量文本，克服单个API调用的token限制。

4. summary_chain 变量：
   - 存储创建的摘要链对象。
   - 这个对象可以用于后续的文档摘要生成。

5. 使用场景：
   - 适合处理长文档或多个相关文档的摘要。
   - 可以处理超出单个API调用限制的大型文本。

6. 注意事项：
   - verbose=True 会产生大量输出，可能会影响性能，通常用于调试。
   - map_reduce 方法可能比简单的摘要方法慢，但能处理更长的文本。

7. 潜在的调整：
   - 可以通过修改 load_summarize_chain 的参数来自定义摘要过程。
   - 例如，可以添加自定义的map和reduce提示模板。

8. 后续步骤：
   - 使用 summary_chain.run(docs) 来生成文档的摘要。
   - 分析输出的详细信息，了解摘要生成的过程。

示例用法：
summary = summary_chain.run(docs)
print("Final Summary:", summary)
'''

In [ ]:
# 运行摘要链并获取输出
output = summary_chain.run(docs)

'''
解释：

1. summary_chain.run(docs):
   - 这行代码执行之前创建的摘要链。
   - docs 是之前使用 RecursiveCharacterTextSplitter 分割的文档列表。

2. 执行过程：
   - 对 docs 中的每个文档块应用 "map" 操作，生成单独的摘要。
   - 然后对这些单独的摘要应用 "reduce" 操作，生成最终的综合摘要。

3. output 变量：
   - 存储摘要链生成的最终摘要结果。
   - 这是一个字符串，包含了整个文档的摘要内容。

4. 执行细节（由于 verbose=True）：
   - 在执行过程中，您应该能看到详细的日志输出。
   - 这包括每个步骤使用的提示和中间结果。

5. 执行时间：
   - 这个过程可能需要一些时间，特别是对于长文档或大量文档块。
   - 执行时间取决于文档的长度、复杂度和API的响应速度。

6. 注意事项：
   - 确保您有足够的API调用额度，因为这个过程可能涉及多次API调用。
   - 对于非常长的文档，可能需要考虑费用和时间成本。

7. 潜在的错误处理：
   - 应考虑添加错误处理机制，以应对可能的API调用失败或其他异常。

8. 后续步骤：
   - 检查 output 的内容，确保摘要质量符合预期。
   - 可能需要进一步处理或分析摘要结果。

9. 可能的改进：
   - 添加进度指示器，特别是对于长文档。
   - 实现异步处理，以提高效率。
   - 保存中间结果，以便在出错时可以从断点继续。

示例后续操作：
print("Summary:", output)
# 或者保存到文件
with open('summary.txt', 'w') as f:
    f.write(output)
'''

In [ ]:
# 将输出分割成单独的摘要
summaries = output.split('\n')

# 遍历并打印每个摘要
for summary in summaries: 
    print('- ' + summary)

'''
解释：

1. output.split('\n'):
   - 使用换行符 '\n' 分割 output 字符串。
   - 这假设每个摘要占据一行。
   - 结果是一个包含多个摘要字符串的列表。

2. summaries 变量:
   - 存储分割后的摘要列表。
   - 每个元素应该是一个独立的摘要或摘要的一部分。

3. for 循环:
   - 遍历 summaries 列表中的每个摘要。

4. print('- ' + summary):
   - 为每个摘要添加一个破折号前缀。
   - 这种格式使输出看起来像一个项目列表。

5. 输出格式:
   - 每行以 "- " 开始，后跟一个摘要。
   - 这种格式提高了可读性，使每个摘要点更加清晰。

6. 用途:
   - 适用于查看多个相关但独立的摘要点。
   - 有助于快速浏览文档的主要内容。

7. 注意事项:
   - 这种方法假设每个摘要都是单行的。如果摘要包含多行，可能需要更复杂的分割逻辑。
   - 空行会被作为单独的项目打印，可能需要额外的处理来去除。

8. 潜在的改进:
   - 添加编号：例如 print(f"{i+1}. {summary}") 用于编号列表。
   - 过滤空行：summaries = [s for s in summaries if s.strip()]
   - 限制摘要长度：print('- ' + summary[:100] + '...') 用于长摘要。

9. 可能的扩展:
   - 将摘要保存到文件中。
   - 对摘要进行进一步的分析，如关键词提取。

示例扩展：
# 保存格式化的摘要到文件
with open('formatted_summaries.txt', 'w') as f:
    for summary in summaries:
        if summary.strip():  # 忽略空行
            f.write(f"- {summary}\n")
'''

This summary is a great start, but lets modify to get only the key points in the summary.

In order to do this we will use custom promopts (like we did above) to instruct the model on what we need. Please note that the prompts format that is used in the notebook is based on flan t5, taken from this [source.](https://huggingface.co/jordiclive/flan-t5-11b-summarizer-filtered?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+was+the+first+structure+to+reach+a+height+of+300+metres.+Due+to+the+addition+of+a+broadcasting+aerial+at+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)

The map_prompt is going to stay the same (just showing it for clarity), but I'll edit the combine_prompt.

In [ ]:
from langchain import PromptTemplate

# 定义map阶段的提示模板
map_prompt = """
Write a ~ 500 word summary of the following text:
"{text}"
"""

# 创建PromptTemplate对象
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

'''
解释：

1. 提示模板定义：
   - map_prompt 是一个字符串，定义了生成摘要的指令。
   - 它要求生成大约500字的摘要。
   - "{text}" 是一个占位符，将被实际的文本内容替换。

2. PromptTemplate：
   - 从 langchain 导入的类，用于创建结构化的提示模板。
   - 允许动态插入变量到预定义的模板中。

3. 创建 PromptTemplate 对象：
   - template=map_prompt：使用上面定义的提示字符串作为模板。
   - input_variables=["text"]：指定模板中的变量。这里只有一个变量 "text"。

4. map_prompt_template：
   - 这个对象可以用于生成具体的提示。
   - 使用时，可以通过 map_prompt_template.format(text=some_text) 来生成完整的提示。

5. 用途：
   - 这个模板将用于 map_reduce 摘要链的 "map" 阶段。
   - 它将应用于文档的每个分块，生成初步摘要。

6. "~500 word" 指示：
   - 给出了期望摘要长度的大致指导。
   - 实际生成的摘要长度可能会有所不同，取决于模型的行为。

7. 注意事项：
   - 500字是一个相对较长的摘要。确保这符合您的需求和模型的能力。
   - 对于较短的文本块，可能需要调整这个字数要求。

8. 潜在的改进：
   - 可以添加更多具体的指示，如"focus on key points" 或 "include main arguments"。
   - 考虑添加格式化指示，如"Use bullet points" 或 "Divide into paragraphs"。

9. 灵活性：
   - 这个模板可以根据需要轻松修改，以适应不同的摘要需求。

示例使用：
text_chunk = "Some long text here..."
formatted_prompt = map_prompt_template.format(text=text_chunk)
print(formatted_prompt)
'''

In [ ]:
from langchain import PromptTemplate

# 定义combine阶段的提示模板
combine_prompt = """
Cover only the key points of the text.
{text}
"""

# 创建PromptTemplate对象
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

'''
解释：

1. 提示模板定义：
   - combine_prompt 是一个字符串，定义了合并摘要的指令。
   - 它指示只覆盖文本的关键点。
   - "{text}" 是一个占位符，将被实际的文本内容（在这种情况下是多个摘要的组合）替换。

2. PromptTemplate：
   - 再次使用 langchain 的 PromptTemplate 类来创建结构化的提示模板。
   - 这允许动态插入变量到预定义的模板中。

3. 创建 PromptTemplate 对象：
   - template=combine_prompt：使用上面定义的提示字符串作为模板。
   - input_variables=["text"]：指定模板中的变量。这里只有一个变量 "text"。

4. combine_prompt_template：
   - 这个对象将用于生成具体的合并提示。
   - 使用时，可以通过 combine_prompt_template.format(text=combined_summaries) 来生成完整的提示。

5. 用途：
   - 这个模板将用于 map_reduce 摘要链的 "reduce" 阶段。
   - 它将应用于前面 "map" 阶段生成的多个摘要，以创建最终的综合摘要。

6. "Cover only the key points" 指示：
   - 强调了最终摘要应该聚焦于最重要的信息。
   - 这有助于确保最终摘要简洁且信息丰富。

7. 注意事项：
   - 这个提示相对简短，给模型留下了更多解释空间。
   - 可能需要根据具体需求进行调整，以获得更精确的结果。

8. 潜在的改进：
   - 可以添加更具体的指示，如 "Synthesize the main themes" 或 "Avoid repetition"。
   - 考虑指定期望的输出格式或长度。

9. 灵活性：
   - 这个模板可以根据需要轻松修改，以适应不同的摘要合并需求。

10. 与 map_prompt_template 的区别：
    - 这个模板更加简洁，因为它处理的是已经摘要化的文本。
    - 重点是整合和提炼信息，而不是从头开始总结。

示例使用：
combined_summaries = "Summary 1... Summary 2... Summary 3..."
formatted_combine_prompt = combine_prompt_template.format(text=combined_summaries)
print(formatted_combine_prompt)
'''

In [ ]:
# 创建自定义的摘要链
summary_chain_key_points = load_summarize_chain(
    llm=sm_llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    # verbose=True
)

'''
解释：

1. load_summarize_chain 函数：
   - 来自 langchain.chains.summarize 模块。
   - 用于创建一个自定义的文档摘要处理链。

2. 参数解释：
   - llm=sm_llm: 
     * 指定使用的语言模型。
     * 使用之前创建的SageMaker端点LLM实例。

   - chain_type='map_reduce':
     * 指定摘要链的类型为"map_reduce"。
     * 这种类型首先对每个文档块生成摘要（map步骤），然后将这些摘要组合成最终摘要（reduce步骤）。

   - map_prompt=map_prompt_template:
     * 使用之前定义的自定义map提示模板。
     * 这将用于生成每个文档块的初步摘要。

   - combine_prompt=combine_prompt_template:
     * 使用之前定义的自定义combine提示模板。
     * 这将用于将多个初步摘要合并成最终摘要。

   - # verbose=True:
     * 这行被注释掉了，如果取消注释，将启用详细输出模式。
     * 详细模式有助于调试，但可能会产生大量输出。

3. summary_chain_key_points：
   - 存储创建的自定义摘要链对象。
   - 这个对象可以用于后续的文档摘要生成。

4. 自定义摘要链的优势：
   - 允许更精确地控制摘要过程的每个阶段。
   - 可以根据特定需求调整map和combine阶段的行为。

5. 使用场景：
   - 适合需要特定格式或内容的摘要。
   - 对于需要强调关键点的长文档摘要特别有用。

6. 注意事项：
   - 自定义提示可能需要多次调整才能获得最佳结果。
   - 确保map和combine提示相互兼容，以产生连贯的最终摘要。

7. 潜在的调整：
   - 可以通过修改提示模板来进一步优化摘要过程。
   - 考虑添加错误处理和重试逻辑。

8. 后续步骤：
   - 使用 summary_chain_key_points.run(docs) 来生成文档的摘要。
   - 比较这个自定义链与默认链的结果。

示例用法：
key_points_summary = summary_chain_key_points.run(docs)
print("Key Points Summary:", key_points_summary)

9. 性能考虑：
   - 自定义提示可能会影响处理时间和API调用次数。
   - 在大规模使用前，建议进行性能测试。
'''

Instead of summarizing all the 30 split documents (chunks), I am using only 15 of them to save time  as it can take few minutes and does not run out of memory on the notebook instance.

In [ ]:
# 运行自定义摘要链并获取输出
output_key_points = summary_chain_key_points.run(docs)

'''
解释：

1. summary_chain_key_points.run(docs):
   - 执行之前创建的自定义摘要链。
   - docs 是之前使用 RecursiveCharacterTextSplitter 分割的文档列表。

2. 执行过程：
   - Map 阶段：对 docs 中的每个文档块应用 map_prompt_template，生成初步摘要。
   - Reduce 阶段：使用 combine_prompt_template 将初步摘要合并成最终摘要。

3. output_key_points 变量：
   - 存储摘要链生成的最终摘要结果。
   - 这是一个字符串，包含了聚焦于关键点的文档摘要。

4. 执行细节：
   - 由于 verbose=True 被注释掉，执行过程不会显示详细日志。
   - 如果需要查看中间步骤，可以取消注释 verbose=True。

5. 执行时间：
   - 这个过程可能需要一些时间，特别是对于长文档或大量文档块。
   - 执行时间取决于文档的长度、复杂度和API的响应速度。

6. 预期输出：
   - 根据 combine_prompt 的指示，输出应该集中于文本的关键点。
   - 摘要可能比之前的版本更加简洁和重点突出。

7. 注意事项：
   - 确保有足够的API调用额度，因为这个过程涉及多次API调用。
   - 对于非常长的文档，考虑成本和时间因素。

8. 错误处理：
   - 当前代码没有显式的错误处理。在生产环境中，应添加try-except块来捕获可能的异常。

9. 后续步骤：
   - 检查 output_key_points 的内容，确保摘要质量符合预期。
   - 可能需要进一步分析或处理摘要结果。

10. 可能的改进：
    - 添加进度指示器，特别是对于长文档。
    - 实现异步处理以提高效率。
    - 保存中间结果，以便在出错时可以从断点继续。

示例后续操作：
print("Key Points Summary:", output_key_points)

# 或者保存到文件
with open('key_points_summary.txt', 'w') as f:
    f.write(output_key_points)

# 比较与之前摘要的差异
print("Difference in length:", len(output) - len(output_key_points))
'''

In [ ]:
# 将关键点摘要分割成单独的要点
summaries = output_key_points.split('\n')

# 遍历并打印每个要点
for summary in summaries: 
    print('- ' + summary)

'''
解释：

1. output_key_points.split('\n'):
   - 使用换行符 '\n' 分割 output_key_points 字符串。
   - 假设每个关键点或摘要段落占据一行。
   - 结果是一个包含多个摘要片段的列表。

2. summaries 变量:
   - 存储分割后的摘要列表。
   - 每个元素应该是一个独立的关键点或摘要段落。

3. for 循环:
   - 遍历 summaries 列表中的每个元素。

4. print('- ' + summary):
   - 为每个摘要片段添加一个破折号前缀。
   - 这种格式使输出看起来像一个项目列表，增强可读性。

5. 输出格式:
   - 每行以 "- " 开始，后跟一个摘要片段。
   - 这种格式提高了可读性，使每个关键点更加清晰。

6. 用途:
   - 适用于快速浏览文档的主要观点或关键信息。
   - 有助于以结构化的方式呈现摘要内容。

7. 注意事项:
   - 这种方法假设每个关键点都是单行的。如果有多行关键点，可能需要更复杂的分割逻辑。
   - 空行会被作为单独的项目打印，可能需要额外的处理来去除。

8. 潜在的改进:
   - 过滤空行：summaries = [s for s in summaries if s.strip()]
   - 添加编号：使用 enumerate() 为每个点添加序号。
   - 限制长度：对于很长的点，可以考虑截断或分行显示。

9. 可能的扩展:
   - 将格式化的关键点保存到文件中。
   - 对关键点进行进一步的分析，如关键词提取或情感分析。

示例扩展：
# 保存格式化的关键点到文件
with open('formatted_key_points.txt', 'w') as f:
    for i, summary in enumerate(summaries, 1):
        if summary.strip():  # 忽略空行
            f.write(f"{i}. {summary}\n")

# 打印关键点数量
print(f"Total number of key points: {len([s for s in summaries if s.strip()])}")
'''

## Summarize a book

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document

# 加载PDF文件
loader = PyPDFLoader("data/book/IntoThinAirBook.pdf")
pages = loader.load()

# 打印页数
print('number of pages: ', len(pages))

# 裁剪开头和结尾部分
pages = pages[28:len(pages)]

# 合并页面内容，并将制表符替换为空格
text = ""
for page in pages:
    text += page.page_content
text = text.replace('\t', ' ')

'''
解释：

1. 导入必要的模块：
   - PyPDFLoader：用于加载PDF文件。
   - Document：langchain的文档模式类，虽然在这段代码中没有直接使用。

2. 加载PDF文件：
   - loader = PyPDFLoader("data/book/IntoThinAirBook.pdf")：创建PDF加载器实例。
   - pages = loader.load()：加载PDF文件，返回一个包含每页内容的列表。

3. 打印页数：
   - 使用len(pages)获取页面数量并打印。

4. 裁剪文档：
   - pages = pages[28:len(pages)]：移除前28页。
   - 这可能是为了去除封面、目录等非正文内容。

5. 合并页面内容：
   - 初始化空字符串text。
   - 遍历pages中的每一页，将其内容（page.page_content）添加到text中。

6. 替换制表符：
   - text.replace('\t', ' ')：将所有制表符替换为空格。
   - 这有助于标准化文本格式，去除可能的排版问题。

7. 注意事项：
   - 确保PDF文件路径正确，否则会引发FileNotFoundError。
   - 裁剪页面（第28页开始）是特定于这本书的，可能需要根据不同的PDF调整。

8. 潜在的改进：
   - 错误处理：添加try-except块来处理文件不存在或无法读取的情况。
   - 内存管理：对于非常大的PDF，可能需要考虑分块处理而不是一次性加载所有内容。
   - 文本清理：可以添加更多的文本清理步骤，如去除多余的空白字符。

9. 后续步骤：
   - 文本现在存储在text变量中，可以用于进一步的处理，如分割成小块、生成摘要等。

示例扩展：
# 基本的文本清理
import re

# 移除多余的空白字符
text = re.sub(r'\s+', ' ', text).strip()

# 打印文本的前500个字符作为预览
print("Preview of the text:")
print(text[:500] + "...")

# 打印总字符数
print(f"Total characters: {len(text)}")
'''

In [ ]:
num_tokens = sm_llm.get_num_tokens(text)

print (f"This book has {num_tokens} tokens in it")

Note that AI21 Summarize model can take upto 40k chunk size, therefore, dividing the book into 30k chunks. 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 创建文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "\t"], 
    chunk_size=20000, 
    chunk_overlap=3000
)

# 将文本分割成文档
docs = text_splitter.create_documents([text])

'''
解释：

1. RecursiveCharacterTextSplitter:
   - 这是一个用于将长文本分割成更小块的工具。
   - "递归"意味着它会按照指定的分隔符列表逐级尝试分割文本。

2. 参数解释：
   - separators=["\n\n", "\n", "\t"]:
     * 指定分割文本的分隔符顺序。
     * 首先尝试用两个换行符分割（段落），然后是单个换行符，最后是制表符。
   - chunk_size=20000:
     * 每个文本块的目标大小（以字符为单位）。
     * 分割器会尽量创建不超过这个大小的块。
   - chunk_overlap=3000:
     * 相邻块之间的重叠字符数。
     * 有助于保持上下文连贯性，特别是在块的边界处。

3. create_documents 方法：
   - 接受一个文本列表（这里只有一个元素，即整本书的文本）。
   - 返回一个Document对象列表，每个对象代表一个文本块。

4. docs 变量：
   - 存储了分割后的文档对象列表。
   - 每个文档对象通常包含文本内容和可能的元数据。

5. 分割的重要性：
   - 允许处理超出模型最大输入长度的文档。
   - 有助于更精确地分析长文档的不同部分。

6. 注意事项：
   - chunk_size (20000) 相对较大，确保它不超过您使用的模型的最大输入限制。
   - chunk_overlap (3000) 也相对较大，这有助于保持上下文，但会增加总处理量。

7. 潜在用途：
   - 为长文档生成摘要。
   - 对文档的不同部分进行独立分析。
   - 实现基于大型文档的问答系统。

8. 可能的后续步骤：
   - 检查分割结果，确保分割是合理的。
   - 对每个文档块进行进一步处理，如生成摘要或提取关键信息。

示例扩展：
# 打印分割后的文档数量
print(f"Total number of document chunks: {len(docs)}")

# 打印第一个文档块的预览
if docs:
    print("\nPreview of the first document chunk:")
    print(docs[0].page_content[:500] + "...")

# 检查每个文档块的大小
chunk_sizes = [len(doc.page_content) for doc in docs]
print(f"\nAverage chunk size: {sum(chunk_sizes) / len(chunk_sizes):.2f} characters")
print(f"Smallest chunk size: {min(chunk_sizes)} characters")
print(f"Largest chunk size: {max(chunk_sizes)} characters")
'''

In [ ]:
# 获取分割后的文档数量
num_docs = len(docs)

# 计算第一个文档的token数量
num_tokens_first_doc = sm_llm.get_num_tokens(docs[0].page_content)

# 打印文档数量和第一个文档的token数
print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

'''
解释：

1. 文档数量计算：
   - len(docs) 返回docs列表的长度，即分割后的文档块数量。
   - 这个数量被存储在num_docs变量中。

2. 第一个文档的token计数：
   - docs[0] 访问分割后的第一个文档对象。
   - .page_content 获取该文档对象的文本内容。
   - sm_llm.get_num_tokens() 计算这段文本的token数量。
   - 结果存储在num_tokens_first_doc变量中。

3. sm_llm.get_num_tokens():
   - 这是之前定义的SageMaker端点LLM实例的方法。
   - 用于计算给定文本的token数量。

4. 打印语句：
   - 使用f-string格式化输出信息。
   - 显示总文档块数和第一个文档块的token数。

5. 这段代码的目的：
   - 验证文本分割的效果。
   - 提供关于分割结果的基本统计信息。
   - 帮助评估是否需要进一步调整分割参数。

6. 注意事项：
   - 文档块数量应该与原文长度和chunk_size设置相符。
   - 第一个文档的token数应该接近（但不超过）之前设置的chunk_size。

7. 潜在的后续分析：
   - 可以计算所有文档块的平均token数。
   - 检查是否有任何文档块超出了预期的大小限制。
   - 根据需要调整RecursiveCharacterTextSplitter的参数。

8. 可能的改进：
   - 计算并显示所有文档块的token数统计（最小值、最大值、平均值）。
   - 检查是否有异常大小的文档块。

示例扩展：
# 计算所有文档块的token数
all_token_counts = [sm_llm.get_num_tokens(doc.page_content) for doc in docs]

print(f"Average tokens per document: {sum(all_token_counts) / len(all_token_counts):.2f}")
print(f"Minimum tokens in a document: {min(all_token_counts)}")
print(f"Maximum tokens in a document: {max(all_token_counts)}")

# 检查是否有超大文档块
max_allowed_tokens = 20000  # 假设这是模型的最大输入限制
large_docs = [i for i, count in enumerate(all_token_counts) if count > max_allowed_tokens]
if large_docs:
    print(f"Warning: Documents {large_docs} exceed the token limit.")
'''

In [ ]:
# 定义map阶段的提示模板
map_prompt = """
"{text}"
"""

# 创建PromptTemplate对象
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

'''
解释：

1. 提示模板定义：
   - map_prompt 是一个简单的字符串模板。
   - 它只包含一个用引号括起来的 "{text}" 占位符。
   - 这是一个非常简单的模板，本质上只是将输入文本原样传递，没有添加任何额外的指令。

2. PromptTemplate：
   - 从 langchain 导入的类，用于创建结构化的提示模板。
   - 允许动态插入变量到预定义的模板中。

3. 创建 PromptTemplate 对象：
   - template=map_prompt：使用上面定义的提示字符串作为模板。
   - input_variables=["text"]：指定模板中的变量。这里只有一个变量 "text"。

4. map_prompt_template：
   - 这个对象可以用于生成具体的提示。
   - 使用时，可以通过 map_prompt_template.format(text=some_text) 来生成完整的提示。

5. 用途：
   - 这个模板将用于 map_reduce 摘要链的 "map" 阶段。
   - 它将应用于文档的每个分块。
   - 由于模板非常简单，它实际上只是将原始文本传递给模型，没有添加任何特定的指令。

6. 特点：
   - 极度简化的模板，没有提供任何具体的指导或约束。
   - 允许模型基于其预训练知识自由处理输入文本。

7. 注意事项：
   - 这种简单的模板可能不会产生非常具体或结构化的输出。
   - 模型的输出将高度依赖于其预训练和默认行为。

8. 潜在的改进：
   - 可以考虑添加一些基本指令，如 "Summarize the following text:" 或 "Extract key points from:"。
   - 如果需要更具体的输出，可以添加格式或内容的指导。

9. 灵活性：
   - 这个简单的模板允许模型发挥最大的灵活性，但可能缺乏对输出的控制。
   - 根据具体需求，可能需要在灵活性和具体指导之间找到平衡。

示例使用：
text_chunk = "Some long text here..."
formatted_prompt = map_prompt_template.format(text=text_chunk)
print(formatted_prompt)

# 输出将只是原始文本被引号包围
'''

In [ ]:
# 创建map阶段的摘要链
map_chain = load_summarize_chain(
    llm=sm_llm,
    chain_type="stuff",
    prompt=map_prompt_template
)

'''
解释：

1. load_summarize_chain 函数：
   - 来自 langchain.chains.summarize 模块。
   - 用于创建一个文档摘要处理链。

2. 参数解释：
   - llm=sm_llm: 
     * 指定使用的语言模型。
     * 使用之前创建的SageMaker端点LLM实例。

   - chain_type="stuff":
     * 指定摘要链的类型为"stuff"。
     * "stuff"方法将所有输入文档合并成一个大的文档，然后一次性处理。
     * 适用于总输入长度不超过模型最大输入限制的情况。

   - prompt=map_prompt_template:
     * 使用之前定义的map提示模板。
     * 这个模板非常简单，只是将原文本用引号括起来。

3. map_chain：
   - 存储创建的摘要链对象。
   - 这个对象将用于处理文档的各个部分（在map阶段）。

4. "stuff" 方法的特点：
   - 简单直接，将所有输入合并后一次性处理。
   - 在这种情况下，由于prompt非常简单，实际上就是让模型直接处理原始文本。

5. 使用场景：
   - 这种设置适合于当您希望模型基于其预训练知识自由处理文本时。
   - 可能用于初步处理或分析文本，而不是生成具体的摘要。

6. 注意事项：
   - 由于prompt非常简单，模型的输出将高度依赖于其默认行为。
   - 确保合并后的文档长度不会超过模型的最大输入限制。

7. 潜在的影响：
   - 这种方法可能导致不一致或不可预测的输出，因为没有给模型明确的指令。
   - 可能需要在后续步骤中进行更多的处理或过滤。

8. 后续步骤：
   - 使用 map_chain.run(docs) 来处理文档。
   - 仔细分析输出，看是否符合您的需求。

9. 可能的改进：
   - 如果需要更具体或结构化的输出，考虑修改map_prompt_template。
   - 可以添加错误处理和输出验证机制。

示例用法：
for i, doc in enumerate(docs[:3]):  # 处理前3个文档作为示例
    result = map_chain.run([doc])
    print(f"Result for document {i}:")
    print(result)
    print("-" * 50)

10. 性能考虑：
    - "stuff"方法对每个文档块单独处理，可能会导致多次API调用。
    - 对于大量文档，考虑批处理或异步处理以提高效率。
'''

In [ ]:
# Make an empty list to hold your summaries
summary_list = []

# Loop through a range of the lenght of your selected docs
for i, doc in enumerate(docs):
    
    # Go get a summary of the chunk
    chunk_summary = map_chain.run([doc])
    
    # Append that summary to your list
    summary_list.append(chunk_summary)
    
    # print (f"Summary #{i+1} - Preview: {chunk_summary[:250]} \n")

In [ ]:
# 创建一个空列表来存储摘要
summary_list = []

# 遍历所有文档块
for i, doc in enumerate(docs):
    
    # 获取当前文档块的摘要
    chunk_summary = map_chain.run([doc])
    
    # 将摘要添加到列表中
    summary_list.append(chunk_summary)
    
    # 打印摘要预览（当前被注释掉）
    # print(f"Summary #{i+1} - Preview: {chunk_summary[:250]} \n")

'''
解释：

1. 初始化摘要列表：
   - summary_list = [] 创建一个空列表，用于存储每个文档块的摘要。

2. 遍历文档块：
   - for i, doc in enumerate(docs): 遍历docs列表中的每个文档块。
   - enumerate() 函数同时提供索引 i 和文档块 doc。

3. 生成摘要：
   - chunk_summary = map_chain.run([doc]) 对每个文档块运行map_chain。
   - map_chain 使用之前定义的简单提示模板处理文档。
   - 由于提示模板非常简单，这实际上可能不是在生成传统意义上的摘要，而是对文本进行某种处理或转换。

4. 存储摘要：
   - summary_list.append(chunk_summary) 将每个处理结果添加到summary_list中。

5. 打印预览（当前被注释掉）：
   - 如果取消注释，将打印每个摘要的前250个字符。
   - 这有助于快速查看处理结果。

6. 注意事项：
   - 这个过程可能会很耗时，特别是如果文档块数量很多。
   - 由于使用的是"stuff"方法，每个文档块都会单独发送到模型，可能导致大量API调用。

7. 潜在的改进：
   - 错误处理：添加try-except块来处理可能的API调用失败。
   - 进度跟踪：添加进度条或定期打印状态更新。
   - 批处理：考虑批量处理文档块以减少API调用次数。

8. 后续步骤：
   - 分析summary_list中的结果，看它们是否符合预期。
   - 可能需要进一步处理这些结果，例如合并或提取关键信息。

9. 可能的扩展：
   - 保存中间结果，以防处理中断。
   - 实现并行处理以提高效率。

示例扩展：
import time

start_time = time.time()

for i, doc in enumerate(docs):
    chunk_summary = map_chain.run([doc])
    summary_list.append(chunk_summary)
    
    if (i+1) % 10 == 0:  # 每处理10个文档块打印一次进度
        print(f"Processed {i+1}/{len(docs)} chunks")

print(f"Total processing time: {time.time() - start_time:.2f} seconds")
print(f"Total summaries generated: {len(summary_list)}")
'''

In [ ]:
# 创建PromptTemplate对象
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

'''
解释：

1. 提示模板定义：
   - combine_prompt 是一个简单的字符串模板。
   - 它只包含一个用引号括起来的 "{text}" 占位符。
   - 这是一个非常简单的模板，本质上只是将输入文本原样传递，没有添加任何额外的指令。

2. PromptTemplate：
   - 从 langchain 导入的类，用于创建结构化的提示模板。
   - 允许动态插入变量到预定义的模板中。

3. 创建 PromptTemplate 对象：
   - template=combine_prompt：使用上面定义的提示字符串作为模板。
   - input_variables=["text"]：指定模板中的变量。这里只有一个变量 "text"。

4. combine_prompt_template：
   - 这个对象可以用于生成具体的提示。
   - 使用时，可以通过 combine_prompt_template.format(text=some_text) 来生成完整的提示。

5. 用途：
   - 这个模板将用于摘要链的 "combine" 或 "reduce" 阶段。
   - 它将应用于合并多个文档块的摘要或处理结果。
   - 由于模板非常简单，它实际上只是将原始文本传递给模型，没有添加任何特定的指令。

6. 特点：
   - 极度简化的模板，没有提供任何具体的指导或约束。
   - 允许模型基于其预训练知识自由处理输入文本。

7. 注意事项：
   - 这种简单的模板可能不会产生非常具体或结构化的输出。
   - 模型的输出将高度依赖于其预训练和默认行为。
   - 在combine阶段，这可能导致不一致或不可预测的最终摘要。

8. 潜在的改进：
   - 考虑添加一些指导性指令，如 "Synthesize the following summaries into a coherent text:" 或 "Combine the key points from these summaries:"。
   - 如果需要更具体的输出格式或内容，可以在模板中添加相应的指示。

9. 灵活性与限制：
   - 这个简单的模板给予模型最大的灵活性，但也可能导致缺乏对最终输出的控制。
   - 根据具体需求，可能需要在灵活性和具体指导之间找到平衡。

示例使用：
combined_text = "Summary 1... Summary 2... Summary 3..."
formatted_prompt = combine_prompt_template.format(text=combined_text)
print(formatted_prompt)

# 输出将只是原始文本被引号包围

10. 与map_prompt_template的关系：
    - 这个combine_prompt_template与之前定义的map_prompt_template非常相似。
    - 在复杂的摘要任务中，通常会对combine阶段使用不同的、更具指导性的提示。
    - 当前的设置可能导致map和combine阶段的行为非常相似，可能需要根据具体需求进行调整。
'''

In [ ]:
from langchain.chains.summarize import load_summarize_chain

# 创建reduce阶段的摘要链
reduce_chain = load_summarize_chain(
    llm=sm_llm,
    chain_type="stuff",
    prompt=combine_prompt_template,
#   verbose=True  # 设置为True可以查看内部工作过程
)

'''
解释：

1. load_summarize_chain 函数：
   - 来自 langchain.chains.summarize 模块。
   - 用于创建一个文档摘要处理链。

2. 参数解释：
   - llm=sm_llm: 
     * 指定使用的语言模型。
     * 使用之前创建的SageMaker端点LLM实例。

   - chain_type="stuff":
     * 指定摘要链的类型为"stuff"。
     * "stuff"方法将所有输入文档合并成一个大的文档，然后一次性生成摘要。
     * 适用于总输入长度不超过模型最大输入限制的情况。

   - prompt=combine_prompt_template:
     * 使用之前定义的combine提示模板。
     * 这个模板将用于指导模型如何生成最终摘要。

   - # verbose=True:
     * 这行被注释掉了。如果取消注释，将启用详细输出模式。
     * 详细模式有助于调试，显示链的内部工作过程。

3. reduce_chain：
   - 存储创建的摘要链对象。
   - 这个对象将用于生成最终的摘要。

4. "stuff" 方法的特点：
   - 简单直接，适合处理较短的文档或文档集。
   - 一次性处理所有输入，可能产生更连贯的摘要。
   - 限制：如果合并后的文档超过模型的最大输入长度，将无法处理。

5. 使用场景：
   - 适合处理已经过初步摘要或较短的文档集。
   - 当您希望生成一个整体性强的最终摘要时很有用。

6. 注意事项：
   - 确保合并后的文档长度不会超过模型的最大输入限制。
   - 对于非常长的文档集，"stuff"方法可能不适用，需要考虑使用其他方法如"map_reduce"。

7. 潜在的调整：
   - 可以通过修改combine_prompt_template来调整摘要的风格或重点。
   - 考虑添加错误处理，以应对可能的输入过长情况。

8. 后续步骤：
   - 使用 reduce_chain.run(docs) 来生成最终摘要。
   - 分析生成的摘要，确保它捕捉了文档的关键点。

示例用法：
final_summary = reduce_chain.run(docs)
print("Final Summary:", final_summary)

9. 性能考虑：
   - "stuff"方法通常比"map_reduce"更快，因为它只需要一次模型调用。
   - 但对于大型文档集，可能需要考虑内存使用和模型输入限制。
'''

In [ ]:
# 使用reduce_chain处理summaries并获取最终输出
output = reduce_chain.run([summaries])

'''
解释：

1. reduce_chain.run():
   - 调用之前创建的reduce_chain来处理summaries。
   - reduce_chain 是使用 load_summarize_chain 函数创建的，类型为 "stuff"。

2. [summaries] 参数:
   - summaries 被放在一个列表中传递给 run 方法。
   - 这里的 summaries 应该是之前在 map 阶段生成的摘要列表。

3. 处理过程:
   - reduce_chain 会将所有的 summaries 合并成一个文本。
   - 然后使用之前定义的 combine_prompt_template 来处理这个合并后的文本。
   - 由于 combine_prompt_template 非常简单（只是将文本用引号括起来），模型会直接处理合并后的文本。

4. output 变量:
   - 存储 reduce_chain 处理后的最终结果。
   - 这个结果应该是对所有summaries的一个综合处理或摘要。

5. 注意事项:
   - 由于使用的是 "stuff" 方法，所有的 summaries 会被一次性处理。
   - 确保合并后的文本长度不超过模型的最大输入限制。
   - 输出的质量和特性将高度依赖于模型的默认行为，因为没有给出具体的指令。

6. 潜在的问题:
   - 如果 summaries 非常长，可能会超出模型的输入限制。
   - 输出可能不够结构化或不符合特定的格式要求，因为没有给出明确的指令。

7. 可能的改进:
   - 添加错误处理，以应对可能的API调用失败或输入过长的情况。
   - 考虑在 combine_prompt_template 中添加更具体的指令，以获得更结构化的输出。

8. 后续步骤:
   - 检查 output 的内容，确保它符合您的需求。
   - 可能需要进一步处理 output，例如分割成段落或提取关键信息。

示例扩展：
# 打印输出预览
print("Output preview:")
print(output[:500] + "..." if len(output) > 500 else output)

# 计算输出的长度
print(f"\nOutput length: {len(output)} characters")

# 如果需要，可以将输出保存到文件
with open('final_summary.txt', 'w') as f:
    f.write(output)

# 简单的输出分析
sentences = output.split('.')
print(f"\nNumber of sentences: {len(sentences)}")
print(f"Average sentence length: {sum(len(s.strip().split()) for s in sentences) / len(sentences):.2f} words")
'''

In [ ]:
# 将输出分割成单独的关键点
key_points = output.split('\n')

# 遍历并打印每个关键点
for key_point in key_points: 
    print('- ' + key_point)

'''
解释：

1. output.split('\n'):
   - 使用换行符 '\n' 分割 output 字符串。
   - 假设每个关键点占据一行。
   - 结果是一个包含多个关键点的列表。

2. key_points 变量:
   - 存储分割后的关键点列表。
   - 每个元素应该是一个独立的关键点或摘要句。

3. for 循环:
   - 遍历 key_points 列表中的每个元素。

4. print('- ' + key_point):
   - 为每个关键点添加一个破折号前缀。
   - 这种格式使输出看起来像一个项目列表，增强可读性。

5. 输出格式:
   - 每行以 "- " 开始，后跟一个关键点。
   - 这种格式提高了可读性，使每个关键点更加清晰。

6. 用途:
   - 适用于快速浏览文档的主要观点或关键信息。
   - 有助于以结构化的方式呈现摘要内容。

7. 注意事项:
   - 这种方法假设每个关键点都是单行的。如果有多行关键点，可能需要更复杂的分割逻辑。
   - 空行会被作为单独的项目打印，可能需要额外的处理来去除。

8. 潜在的改进:
   - 过滤空行：key_points = [point for point in key_points if point.strip()]
   - 添加编号：使用 enumerate() 为每个点添加序号。
   - 限制长度：对于很长的点，可以考虑截断或分行显示。

9. 可能的扩展:
   - 将格式化的关键点保存到文件中。
   - 对关键点进行进一步的分析，如关键词提取或情感分析。

示例扩展：
# 保存格式化的关键点到文件
with open('formatted_key_points.txt', 'w') as f:
    for i, point in enumerate(key_points, 1):
        if point.strip():  # 忽略空行
            f.write(f"{i}. {point}\n")

# 打印关键点数量
print(f"Total number of key points: {len([p for p in key_points if p.strip()])}")

# 如果需要限制每个点的长度
max_length = 100
for point in key_points:
    if point.strip():
        print(f"- {point[:max_length]}{'...' if len(point) > max_length else ''}")
'''

## Clean Up
*NOTE:* Please make sure to delete the endpoint, if you are not using it, as it will incur charges. 

In [ ]:
# # Specify the name of your endpoint
# endpoint_name_llm="summarize"

# # # Create a low-level SageMaker service client.
# sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
                        
# # # Delete endpoint configuration
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name_llm)

# # Delete endpoint
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name_llm)